In [1]:
import unittest

custom_username_env = 'maiotradeuser'
custom_pw_env = 'maiotradepw'
custom_clientid_env = 'maiotradeclientid'

# Unit Tests

## Test Configuration

Configure the Ameritrade client to use the username, password and client id stored in environment variables.

In [2]:
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import os

os.environ['ameritradeuser'] = 'test_user'
os.environ['ameritradepw'] = 'test_pw'
os.environ['ameritradeclientid'] = 'client_id'

class TestConfiguration(unittest.TestCase):
    def test_config_default(self):
        class_under_test = amc.AmeritradeRest()
        self.assertEqual(class_under_test.username, 'test_user')
        self.assertEqual(class_under_test.password, 'test_pw')
        self.assertEqual(class_under_test.client_id, 'client_id')  
        self.assertEqual(class_under_test.consumer_key, 'client_id'+'@AMER.OAUTHAP')
    
    def test_config_parms(self):
        class_under_test = amc.AmeritradeRest('ameritradeuser', 'ameritradepw', 'ameritradeclientid')
        self.assertEqual(class_under_test.username, 'test_user')
        self.assertEqual(class_under_test.password, 'test_pw')
        self.assertEqual(class_under_test.client_id, 'client_id')
        self.assertEqual(class_under_test.consumer_key, 'client_id'+'@AMER.OAUTHAP')
        

## Account Level Functions

In [3]:
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import os

class TestAccountLevelFunctions(unittest.TestCase):
    def test_mask_account(self):
        class_under_test = amc.AmeritradeRest(custom_username_env, custom_pw_env, custom_clientid_env)
        self.assertEqual(class_under_test.mask_account('12345678'), '#---5678')
        class_under_test.account_mask = "*****"
        self.assertEqual(class_under_test.mask_account('12345678'), '*****678')
        

## Stock Information Functions

In [4]:
import importlib
import ameritrade_functions as amc
importlib.reload(amc)

class TestStockInformationFunctions(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.class_under_test = amc.AmeritradeRest(custom_username_env, custom_pw_env, custom_clientid_env)
        
    def test_get_daily_price_history(self):
        price_histories = self.class_under_test.get_daily_price_history('AAPL', '2022-01-03')
        display(price_histories)
        self.assertEqual(len(price_histories), 253)
        self.assertEqual(price_histories.date.min().strftime('%Y-%m-%d'), '2021-01-04')
        self.assertEqual(price_histories.date.max().strftime('%Y-%m-%d'), '2022-01-03')
        self.assertEqual(price_histories.ticker.unique()[0], 'AAPL')
        
    def test_get_price_histories(self):
        price_histories = self.class_under_test.get_price_histories(['AAPL', 'GOOG'], '2022-01-03', silent=True)
        self.assertEqual(len(price_histories), 506)
        self.assertEqual(price_histories.date.min().strftime('%Y-%m-%d'), '2021-01-04')
        self.assertEqual(price_histories.date.max().strftime('%Y-%m-%d'), '2022-01-03')
        self.assertEqual(len(price_histories.ticker.unique()), 2)
        


In [5]:
cut = TestStockInformationFunctions()
cut.setUpClass()
cut.test_get_price_histories()

# Integration Tests

Requires you to have a developer account with Ameritrade (https://developer.tdameritrade.com).

These settings are configured via environment variables.

## Test Authenticated Functions

In [11]:
import importlib
import ameritrade_functions as amc
importlib.reload(amc)

class TestAuthenticated(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.class_under_test = amc.AmeritradeRest(custom_username_env, custom_pw_env, custom_clientid_env)
        cls.class_under_test.authenticate()
        
    def test_authentication(self):
        self.assertGreater(len(self.class_under_test.authorization), 0)
        
    def test_get_access_token(self):
        self.assertIsNotNone(self.class_under_test.get_access_token())
        
        
        with self.assertRaises(RuntimeError) as cm:
            amc.AmeritradeRest(custom_username_env, custom_pw_env, custom_clientid_env).get_access_token()
            
    def test_get_accounts(self):
        self.class_under_test.get_accounts()
        self.assertIsNotNone(self.class_under_test.account_data)
    def test_parse_accounts(self):
        self.assertGreater(len(self.class_under_test.parse_accounts()), 0)
        
    def test_get_positions(self):
        self.assertGreater(len(self.class_under_test.get_positions()), 0)
        
    def test_parse_portfolios_list(self):
        self.assertGreater(len(self.class_under_test.parse_portfolios_list()), 0)

# Test Runner

In [12]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_mask_account (__main__.TestAccountLevelFunctions) ... ok
test_authentication (__main__.TestAuthenticated) ... ok
test_get_access_token (__main__.TestAuthenticated) ... ok
test_get_accounts (__main__.TestAuthenticated) ... C:\Users\HP-Z440\anaconda3\envs\ml4t\lib\site-packages\pandas\core\dtypes\common.py:605: ResourceWarning: unclosed <socket.socket fd=2396, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 51134), raddr=('127.0.0.1', 51129)>
  def is_excluded_dtype(dtype) -> bool:
ok
test_get_positions (__main__.TestAuthenticated) ... ok
test_parse_accounts (__main__.TestAuthenticated) ... ok
test_parse_portfolios_list (__main__.TestAuthenticated) ... ok
test_config_default (__main__.TestConfiguration) ... ok
test_config_parms (__main__.TestConfiguration) ... ok
test_get_daily_price_history (__main__.TestStockInformationFunctions) ... 

,open,high,low,close,volume,ticker,date
0,133.520,133.6116,126.760,129.41,143301890,AAPL,2021-01-04
1,128.890,131.7400,128.430,131.01,97664898,AAPL,2021-01-05
2,127.720,131.0499,126.382,126.60,155087970,AAPL,2021-01-06
3,128.360,131.6300,127.860,130.92,109578157,AAPL,2021-01-07
4,132.430,132.6300,130.230,132.05,105158245,AAPL,2021-01-08
...,...,...,...,...,...,...,...
248,180.160,181.3300,178.530,179.29,79144339,AAPL,2021-12-28
249,179.330,180.6300,178.140,179.38,62348931,AAPL,2021-12-29
250,179.470,180.5700,178.090,178.20,59773014,AAPL,2021-12-30
251,178.085,179.2300,177.260,177.57,64062261,AAPL,2021-12-31


ok
test_get_price_histories (__main__.TestStockInformationFunctions) ... ok

----------------------------------------------------------------------
Ran 11 tests in 11.705s

OK
